# Position and number of  detectors for the grating spectrograph

In [ ]:
from nbtemplate import display_header, get_path
display_header('Placethedetector.ipynb')

## Goal

AXIS will have an imaging detector at the location of the direct mirror focus. The same detector can also be used detect the zerothe order when the gratings are in the place, but the diffracted signal needs a separate detector array that is located 30-50 cm from the focal point.

We design the spectrometer with critical angle transmission (CAT) gratings, which are operated with a certain blaze angle. Most of the dispersed signal is found close to twice the blaze angle in a region called the blaze peak. The width of this peak depends on the grating properties (such as the dimensions of the grating bars) and the distribution of blaze angles (due to the finite size of the gratings and the finite size of the mirror PSF not all rays hit the gratings at the blaze angle chosen in the design).

In this notebook, I simulate our fiducial design for the spectrometer and analyze which fraction of the photons can be detected for a detector of a given size and position.

In the last section, I also look at the signal seen in the zeroth order for a range of energies.

In [ ]:
import sys
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.table import Table, QTable, join
import astropy.units as u
import marxs
from marxs import visualization

from marxs.source import PointSource, FixedPointing, JitterPointing
from marxs.analysis import resolvingpower_from_photonlist
from marxs.simulator import Sequence

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from marxs.missions.AXIS import AXIS as axis

In [ ]:
wave = np.arange(12., 50., .1) * u.Angstrom
energy = wave.to(u.keV, equivalencies = u.spectral())
flux = np.ones(len(wave))[1:] / np.abs(np.diff(energy))

mysource = PointSource(coords=SkyCoord(0., 0., unit='deg'),
                       energy=Table({"energy": energy[::-1][1:], 
                                     "fluxdensity": flux[::-1] / u.cm**2 / u.s}),
                       flux=1. / u.s / u.cm**2)
fixedpointing = FixedPointing(coords=SkyCoord(0., 0., unit='deg'))
photons = mysource.generate_photons(1e5 * u.s)
photons = fixedpointing(photons)

In [ ]:
pl = axis.PerfectAXIS()

photons = pl(photons)
photons = photons[np.isfinite(photons['order'])]

## Locating the detectors for the dispersed gratings

In [ ]:
# Definition of circular detector in code is not centered on zero order.
# However, for the plot, we should definitely do that
zeropos = np.mean(photons[photons['order'] == 0]['detcirc_phi'])

fig, axes = plt.subplots(ncols=2, figsize=(10, 4))

out = axes[0].hist(photons['detcirc_phi'] - zeropos, weights=photons['probability'],
                   bins=50, range=[-.2, .1])

out = axes[1].hist(photons['detcirc_phi'] - zeropos, weights=photons['probability'],
                   bins=50, range=[-.16, -0.06])


for ax in axes:
    ax.set_ylabel('Number of photons / bin')
    ax.set_xlabel(r'$\varphi$ (rad)')

Using a simulation with a flat input spectrum (in wavelength space), this figure shows the angular distribution of photons. The x-coordinate of the plot is the angle measured on a cylindrical detector along the Rowland circle. Zero order located at $\varphi=0$, negative orders are seen to the left, and positive order to the right. The blaze peak is the high peak around $\varphi=-0.1$. The smaller peaks around $\varphi=+/- 0.04$ are dominated by low orders (-2, -1, +1, +2) at higher energies where Si becomes transparent to X-rays and the gratings for pure Si gratings. For e.g. Pt coated gratings more of those photons will be redistributed into the blaze peak. 

In [ ]:
photons['wave'] = photons['energy'].to(u.Angstrom, equivalencies=u.spectral())
wavelim = np.arange(12., 49, 12)
px = [photons['proj_x'][(photons['wave'] > wavelim[i]) & (photons['wave'] < wavelim[i+1])] for i in range(len(wavelim)-1)]
weights = [photons['probability'][(photons['wave'] > wavelim[i]) & (photons['wave'] < wavelim[i+1])] for i in range(len(wavelim)- 1)]
labels = ['{}-{} nm'.format(wavelim[i]/10, wavelim[i+1]/10) for i in range(len(wavelim) -1)]

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
out = ax.hist(px, weights=weights, bins=np.arange(-400, 800, 30), histtype='barstacked', label=labels)
ax.legend()
ax.set_ylabel('Number of photons')
ax.set_xlabel('Distance from focal point [mm]')
fig.savefig(get_path('figures')+ '/detectorplacement.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/detectorplacement.pdf', bbox_inches='tight')


This is a very similar to the plot above. This time, the x-axis of the plot is simply the distance of the detected position projected onto the focal plane. The blaze peak is wider for photons with a longer wavelength. From this figure, we can already read off that we need to cover about the region from 400 to 600 mm measured from the zeroth order to collect the bulk of the dispersed signal.

In [ ]:
fig = plt.figure(figsize=(8, 5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for i in np.arange(-9, 1):
    porder = photons[photons['order'] == i]
    out = ax1.hist(porder['proj_x'], weights=porder['probability'], bins=np.arange(-400, 800, 25.), 
                   label=f'{i}', histtype='step')
    out = ax2.hist(porder['wave'], weights=porder['probability'], bins=np.arange(5., 50., 1.),
                  histtype='step')

ax1.legend(loc='upper left')
ax1.set_ylabel('Number of photons')
ax1.set_xlabel('Distance from focal point [mm]')
ax2.set_xlabel('wavelength [Ang]')
ax2.set_ylabel('Number of photons')

*left:* Same plot as above, but separated by dispersion order. *right:* Most wavelength will be dispersed into more than one order and each order is relevant for a range of wavelengths.

## How many detectors will we need?

The plots above show that the blaze peak is pretty sharp and no more than about 200 mm wide. Yet, a large fraction of the photons can in fact we detected with a shorter detector strip, which would reduce cost, mass, power requirements, and data rate at the expense of effective area. Many considerations contribute to the total effecttive area (the fraction of the aperture covered with gratings and if the gratings are coated are two important ones), but those do not change the shape of the blaze peak. So, we can study the impact that a shorter strip of detectors has on the effective area in general.

In [ ]:
det_length = np.arange(10, 301, 10)
fraction_on_detector = np.zeros_like(det_length)

In [ ]:
fig, ax = plt.subplots()

for i in range(len(wavelim) -1):
    pind = (photons['order'] < -2) & (photons['wave'] > wavelim[i]) & (photons['wave'] < wavelim[i+1])
    h, xe = np.histogram(photons['proj_x'][pind], 
                         weights=photons['probability'][pind],
                         bins=np.arange(0, 1000))
    hcumsum = np.cumsum(h)
    for j, l in enumerate(det_length):
        fraction_on_detector[j] = np.max(hcumsum[det_length[j]:] - hcumsum[:-det_length[j]])
    ax.plot(det_length, fraction_on_detector / fraction_on_detector.max(),
            label='{}-{} nm'.format(wavelim[i] / 10, wavelim[i + 1] / 10))
                
ax.legend()
ax.set_xlabel('length of detector array')
ax.set_ylabel('Fraction of dispersed photons')

This plot shows the fraction of the dispersed photons that a detector array of a certain dimension will catch, normalized to an array with 300 mm - enough to include the far wings of the blaze peak. This plot does not account for details like the placement of chip gaps, but can be used to choose how many CCDs the AXIS detector for the dispsered signal should have. There are some bumps and non-linearities in the plot. Those come from the fact that at most wavelengths, more than one order contributes to the blaze peak (see plots above, e.g. at 20 Å order -5, -6, and -7 are significant). The positions of the orders change with wavelength according to the grating equation $n \lambda = d \sin \theta$ where $n$ is the order, $\lambda$ the wavelength, $d$ the grating constant and $\theta$ the dispersion angle. For a single given wavelength, there are thus steps in the fraction of detected photons, where a large detector catches all three orders. For a slightly shorter detector, the fraction will drop as one order drops of the detectors, but then shortening the detector further will not have an impact on the fraction of photons detected, until it becomes so short that another order drops off and only one order fits on the detector.

The plot avove averages those steps over a range of wavelengths (the legend in the plot), which leads to slightly smoother curves, but the blue curve still looks closer to the green than to the orange curve, which describes the wavelength range between the blue and green curve. Selecting a different input spectrum or changing the exact boundaries of the bands, moves those curves around a bit.

However, the general trend is robust. We can see that a detector with a width of order 100 mm will only catch about 50-70% the photons. For scale, the CCD-ID 94's planned for Arcus are about 50 mm long, so this would be just 2 CCDs. There is little reason to use more than 4 CCDs of this type, because a 200 mm long detector covers essentially the entire blaze peak. (For comparison, Arcus uses 8 CCD-ID 94 detectors per camera. The Arcus gratings are only 4 μm deep and thus the blaze peak is wider.) Of couse, AXIS might choose a different detector type, but it should still plan to cover about 150-200 mm of length to catch most of the blaze peak.

**Looking at this, I will baseline 4 CCD-ID 94 detectors. AXIS may adopt different detectors with different shapes, so the exact position of chip gaps etc. might change but I do have to put some assumptions in right now, just to get the simulations going.** In contrast, the direct imaging detector is currently simulated as a single square detector. In practice, we will want to use the same type of detector for both applications, but for the simulations at this stage, I just need some nominal layout so that images and 3D layout look realistic. 